# Log of work on parking provision project

## w/c 25/05/2020
- OS mastermaps tour meeting with REDACTED, REDACTED, REDACTED
- Os getting data ready; converting to geopackage from GML for Ealing and Cornwall
- got advice from SK on geospatial python modules; rasterio, fiona, geopandas etc and environment set up
- advised to use conda
- set up conda environment parkprov

## w/c 01/06/2020
- finished project proposal and handed in
- got data for Ealing, Cornwall geopackage of OSMM topo layer and others
- got data for Ealing and built up areas of Cornwall aerial photography tif files; 1km grid per tile
- exploring AP tif files and OSMM gpkg files in python; rasterio
- reading up on OSMM AddressBasePremium & Topography layers and AP technical specifications
- reading QGIS tutorials, geospatial python module tutorials

## w/c 08/06/2020
- realsied that TopographicLine layer is short line segments, not boundaries of polygons - so not useful
- exploring AdressBase Premium - realised that to match UPRN to corresponding TOIDs in TopoArea ayer I actually need AddressBase PLus - emailed IB and OB (15/06/2020)

## w/c 15/06/2020
- used QGIS successfully to clip Ealing TopograhicArea and TopographicPoint layer to just those features within the AP 1980 grid file (with my flat in). This is much smaller size data and so a lot easier to deal with during exploration.
- identified relevant features in the TopographicArea layer
    - Theme: Land (includes private gardens), Buildings (includes private residence)
    - DescriptiveGroup: Path (includes paths on multiple residence property), Roadside (pavements), Road Or Track (road or track suitable for vehicle)
- get AddressBase Plus gpkg - Ian Bennet arranged the dwonload for of the whole of GB - 16GB when unpacked


- use AddressBase Premium ApplicationCrossReferenceMember to get UPRN to TOID lookup, source code for TOIDs in TopographicArea layer is 7666MT. Got a list of UPRNs with postcodeType Small and their 7666MT TOIDs, 135979 rows.

- create some property sized images to run image processing on!

## w/c 22/06/2020
- continued to work on getting list of appropriate gardens

## w/c 29/06/2020
- matched residential building toids with roadside garden toids that lie within the same land reigistry land parcel. 
- created a lookup between land registry INSPIREID and rd_poly_toid and gdn_poly_toid (both from the TopographicArea kayer)
- had a catch up with Steve Kingston to discuss how to cut the garden vector shapes out of the raster image
- found earthpy and rasterio methods to do this
- wrote jigsaw function that will take a list of garden toids and cut them out of the rasterimage, and then save as a tif file. Ready for image processing. DONE
    - may need to add indication of which edge the roadside adjoins to
    - may need rotating? put roadside edge south
- meeting with RC discussed problem of how to check if a car sized polygon/rectangle will fit into another polygon. There are theorems on the largest rectangle inscribed in a polygon. Don't want largest but if I found the largest then could compare it to a parking space sized rectangle
- RC agreed that it would be useful to have the edge that the roadside adjuncts to visible on the raster image, say in magenta so it does not interfere with the green space detecton algorithm.
- assuming the green space algorithm will classify each pixel as green space or not green space then could redcue image to black for a pixel that could be parked on, then run an algorithm that checks if a car parking space size/number of these pixels exists. Would need to be adjacent to the road though.

## w/c 06/07/2020
- Meeting with SK and IB. 
    - use ONS greenspaces research to decide which vegetation/manmade spectrum to use
    - suggested using the RGBI near infra red imagery, 50cm resolution (Os got me this for Ealing)
    - want to process image to segment it into polygons of manmade vs vegetation; then process the manmade polygons to see if a car will fit.
    - would be good to include information about which edge is adjacent to the roadside; as RC agreed. But SK suggested better to have this as a line segment, with a buffer, so a narrow polygon; obtain by doing another inner join on the roadside shapes and the gardens.
    - Colleague of SK Sheung Zhan has done some work using the Minkowski Difference to fit a polyhedron/gon into another polyhedron/gon. Not read up on it yet but seems to try different angles of orientation, calculate the Minkowski difference, presume this is some kind of measure of overlap and then move on to try another angle -perhaps.
    - having the linestrig of the roadside edge of the garden indicates the access edge to the front garden potential parking space. Also gives orientation to the road, which may be useful in chopping of narrow access to rear gardens from detatched/semidetached properties.
    - as you can rasterise a vector, you can vectorise a raster. So could rasterise the roadside edge linestring and include it in the image - but think this will make things more complicated for the image processing.
    - discussed clustering of property images to based on some manual classification of a set of representative images. manual classification would decide if this property would fit a car in the front garden, then group together with similar properties (by size and shape) - would this be best done on just the gardens, or with the building, land extent as well?
- contacted Chris Bonham to talk about the greenspaces, when I've read it a bit. 
- download code and read up on the garden spaces ONS project: Will not be able to use their tool as is. Rather use their research as the foundation for which vegetation spectrum to use in my image recognition
- talked to dad about the Minkowski difference A vector subtraction B, to see if B fit in A. Not sure if it's any better than using geopandas contains function, to check if B fits in A. In both cases will have to write a program to align the rectangle to the roadside edge and then check if it fits, if it doesn't try a different position/orientation of the rectangle. Key will be to make the first guess a good guess! possibly middle, end on. Or I need to try it on a bunch of gardens and collect the most frequent positions/angles that occur. Consider 3 positions along the roadside edge, left, middle, right. Starting short end on, 0 degrees to the roadside edge, treat the roadside edge as the positive x-axis. Then from the left position try angles turning inward, tilting on the bottom right corner, 30, 60 90. From the right do the same tilting on the bottom left corner, 0, -30, -60, -90. Or smaller steps, 0, 15, 30, 45, 60, 75, 90.


- read greenspaces code: looked at it, it's horrible
- read Sheung Zhou paper / or just read up on Minkowski Difference: understand Mink diff now


## w/c 13/07/2020

- read up on scikit-image https://datacarpentry.org/image-processing/aio/index.html
- 14/07/2020 chat with Chris Bonham on the green spaces code
- 15/07/2020 catch up with OS, SK, IB.
- deduplicated gdn, and rs images - so geodataframes only contain the unique gardens in gdn, that intersect with the unique roadside polygons in rs. 
- Dropped 7 troublesome gardens that had more than one roadside polygon intersection
- trying to collect linestrings of roadside edges of the garden polygons using overlay. Must set keep_geom_type = False, to get linestrings from polygon types. 
- also created a set of roadside polygons with 50cm buffer and used overlay to created roadside fat edge polygon. 

- look at Sheung Zhou code; thi sis written in java. HAve established that I woudl still need to write a program to align and try out the parking rectangle in different positions, orientations, then calculate the Mink Diff. So I could do this using a shapely/gpd function 'contains' to see if the rectangle fits 
- Need to write up stuff so far: plan, development, thoughts, background, images... Write up in word, so I can use the Mendeley citation plugin; and format pictures properly. Conversion of python notebook to PDF did not go so well.
- collect garden shapes from the RGBI imagery


## w/c 20/07/2020

- worked on getting the LineStrings of the roadside edge for each garden
- had to merge lines in multipologons that were created - mostly these were going around corners, but some apparently stright pieces
- buffered the linesting with flat caps to inspect
- then buffered the linestring to 9m with round caps and overlaid with gardens to crop gardens at 9m perpendicular to the roadside linestring. Overaid individual pairs of buffered roadside linestring and its garden to avoid picking up space from adjacent gardens
- wrote this file of new polygons - the cropped gardens to csv, with garden TOID for ID
- wrote roadside linestrings to file
- cut out cropped gardens images from AP RGBI and RGB
- contaced Chris Bonham re HSV transformation, forwarded to Ian Grimstead to get pointer to the code for resizing and image - eg taking the 50cm image and swapping each pixel for 4 the same to make it 25cm 'resolution': use OpenCV resize function; green spaces example line 332, class OrdnanceSurveyMapLoaderWithInfraRed here: https://github.com/datasciencecampus/green-spaces/blob/develop/green_spaces/image_loaders.py
- read papers on remote sensing methods
- experimented with earthpy and cv2 methods for calculating NDVI ppp12_NDVI.ipynb
- now I have a single number for each pixel to threshold the image on.
- plotted histograms of combined NDVI values for 1000 imgaes, range is pretty close to [-0.1,0.1]
- so I can just just 0 as a cut off threshold, and let >0 be veg
- investigated occurances of 'natural' zero value NDVI, as opposed to artifical white area which I'd used to mask the non-garden space. Due to gardens not being aligned vertically and horizontally. Natural zero NDVI values occur when the NIR and Red channel values are equal. In 1000 images, after removal of the white values, 3% of the remaining NDVI values were 'natural'.





## w/c 27/07/2020

- realised my NDVI calculation using the cv2 method was incorrect as I had failed to understand how cv2 reorders the channels. It expects to read in RGB in that order and then converts to BGR by swapping the first and last channels. This means that the CIR 50cm images are stored by cv2 as Green, Red, NIR in that order. I had calculated the normalised difference of Green & Red, instead of NIR & Red. Now, corrected.
- also realsied why my earlier calculations the NDVI range was wrong, outside [-1, 1]. This was because I had not converted the values from int to float. Still don't understand why doing the calculaiton with ints out the range in roughly [0,9], must be something to do with how the numbers are stored in bits. 
- segemented image into external area, vegetation and manmade surface using combination of NIR,R,G values and NDVI values. Created 3 arrays, each with 255 in only one channel corresponding to external (blue), veg (green), manmade surface (red), then stacked and reshaped them to (3, height, width) shape as rasterio uses. Then assigned meta data from original image to these data and wrote the new image tiff. Seemed to work as expected :o) !!!
- now I can use these colour bands as a mask to vectorise!!! expect to have some polygons with holes in, and some multiple polygons. Considered converted multiple polygons to a multipolgon - but think this is unhelpful. By eye most holes are small and where there are multipolygons and there is usually a main one a lot bigger than the others. Consider consuming small holes (say under 1m2) and keeping only the largest polygon - that intersects with the roadside edge. 
- can definitely drop polygons under 11.52m2 - although this assumes that 2 manamde surface polygons in the same garden are legitimately disconnected by vegetation. That is, there is no reasonable way to join them together by discarding the vegatation areas separating them. 
- vectorised some individual polygons and checked all aligned with original cropped images 


- decided to crowd source maual labelling of cropped RGB images. Worked out how to auto load images into excel to make it easy for labellers to look at images and type a code. 1 = CAR PRESENT, 2 = YES, parking (but no car in image), 3 = NO parking, 4 = not sure either way. I managed to label about 200 images in 30mins.

NEXT
- vectorise all manmade surface; drop ones with area less than 11.52m^2, append to single df (with potentially multiple rows per toid) write to csv. First drop 
- organise manual labelling
- fitting rectangles -read up on shapely
- REMEMBER IDEAS from catchup with Riccardo in notebook, and links copied into ppp13 at the end

- going to end up with a class per toid - yes/no parking possible.

Plan 

- apply threshold single numeric threshold for NDVI
- create vector polygons from the manmade surface pixels, possible problems:
    - expect to collect multi polygon of all manmade surface polygons of pixels
    - expect holes, jagged edges,
    -  could create convex hull around biggest polygons that intersect with roadside edge linestring; smooth polygon edges somehow
    - roadside edge linestring may not intersect properly cos of pixel jaggedness 
        - although when I created the rasters I selected any pixel touching to be included, so perhaps this means the line string will be inside the pixel boundary. Still there may be issues at the edge anyway.
- intersect with original cropped garden polygon, to dump non-garden space pixels that may have been included
- align roadside edge linestring
- try to fit car rectangle polygon in different orientations, starting positions.


## w/c 03/08/2020

### Continued with ppp13
Throughout all shape manipulations always keep TOID reference
- vectorised all manmade surfaces - vms object, and tidied by doing the following
    - drop vms polygons with area under 12.5 sqm - since each polygon area is a mutliple of 0.25sqm this is the closest whole number rounded up from 11.52 sqm (2.4 * 4.8)
    - drop vms polygons that do not intersect with a roadside linestring - corresponding by TOID
    - calculate length of intersection of of rsls with corresponding vms and drop those where rsls edge is less than 2.4m. We are taking this as the width of a car, so if rsls edge is less a car cannot enter 
    - find area of holes in polygins and fill those less than 1 sqm (4 pixels)
- reoriented mammade surface polygons so that the holes, represented by interior linear rings are plotted correctly when plotting from geopandas (vis descartes & matplotlib). If the holes are not correctly orieted then descartes does not pass the hole information to matplotlib and the plot fills in the holes. They still show on the shapely plot, which is how I noticed.
- plotted some images of RGB 25cm resolution background, VMS, RSLS and cropped garden polygon edge. These highlight problems where car may turn into available space - so a rectangle with one edge aligned to the roadside will not fit, but would need to be pivoted on one corner. 
- calculated area of VMS and looked at distribution, dropped VMS polygons less than or equal to 12.5 sqm. Probably should've only dropped those LESS THAN 12.5 sqm. Although for a car to fit in 12.5 sqm the area still has to be precisely the right shape - ie 2.5m by 5m, and oriented with axes. Seems unlikely.
- intersected VMS with corresponding rsls to get sections of linestring that overlap manmade surface. This resulted in multilinestrings as well. Applied linemerge to combine multilinestring that were actually continuous. Then exploded all to create df of manmade surface roadside linesgtrings with multiple rows per TOID if a vms had more than one potential drive entrance.
- Investigated duplicate TOIDs in VMS df. Plotted exmaple shows a corner poperty with multiple manmade surface areas around the building edge and in the rear garden. Rear garden is include on corner properties since the  roadside polygon wraps around corners!
- using the exlpoded rsls mms sections - dropped any less than 2.4 m in length, created **rsls24**. Looked at distribution of lengths first, plotted histograms.
- filtered the VMS polygons for only those TOIDs in rsls24
- looked at another example of multiple VMS polygon for a TOID, now including only manmade surface areas with a roadside linestring intersection greater than or equal to 2.4m. Example again shows a corner property
- filled holes <= 1sqm. First extracted interior linear rings from VMS polygons as a new column in the **vms**
- from these created new gdf **hp** of TOID, interior linearring list, created polygons from the holes and found their areas. Holes in polygons are representated by 'interior linear rings' which has length but no area, it is not a polygon. Plotted histograms of the distribution of the hole areas and decied to fill holes <= 1sqm 
- collected **vms** and **hp** both gdfs of polygons and TOIDs into one gdf, **p2d** for 'polygons to dissolve'. Then created **dhvms** (dissolved holes vectorised manmade surfaces) by dissolving by TOID. Where there was no hole to combine the vms polygon remained unchanged. The *dissolve* method combines polygons into one new polygon. (though if they do not touch I guess it will make a multipolygon)
- checked an example TOID that had 2 holes one < 1 sqm and one > 1sqm. As expected the smaller hole is filled and the larger hole remained. 
- did a nice plot of remaining vms polygons and their intersecting rsls edges with RGB as background. Shows the potential parking areas and access edge. Can see some propoerties not included - these have already been dropped for one of the reasons in the filtering above
    - manmade surface area too small
    - roadside edge intersection too narrow
- nicely highlights problems mostly to do with the segmentation and NDVI values.
    - tree canopy obscures manmade surface
    - shadow may be segmented as veg or non-veg

## No Parking COUNT

Original number of TOIDS = 1328

No parking count (number of TOIDs)
- (1329 - 1290) = 39 vectorised manmade surfaces too small (less than or equal to 12.5 sqm)
- (1290 - 1158) = 132 vectorised manmade surfaces do not intersect with roadside linestring 
- (1158 - 970) = 188 vectorised manmade surfaces have roadside edges less than 2.4m

Total no parking so far: 359

Previous try with 2m cut off
- (1158 - 1029) = 129 vectorised manmade surfaces have roadside edges <= 2m



### ppp15

First approach to fitting rectangles is to align short side with rsls and try to fit perpendicular to roadside.
- use test toid to work through the steps of fitting a rectangle aligned with the rsls
    - extract points from the rsls
    - use these points to created a 2.4 linestring, one short edge of the car rectangle
    - buffer linestring to 4.8 and intersect with vms area to get polygon of overlap
    - calculate proportion of car rectangle that fits, ratio of remaining space
    - create index to determine when to class as parking/no parking possible.

    
- problems with approach 

## w/c 10/08/2020

- 10 working days, 12 including weekends to report deadline: 12:00 21/08/2020
- 14 days including weekend to live presentation: 09:30 24/08/2020 

### continued with ppp15 & fixed invalid polygons in ppp13
#### Monday
- worked on converting pipeline for test rectangle fitting into pipeline to work on a dataframe of linestrings
- first thought to iterate by TOID then linestring, but this is not practical
- second try; to iterate on through gdf of linestrings, exploded so no multilinestrings
- also exploded the multipolygons that came out of the dissolve


#### Tuesday
- discovered invlaid polygons went I tried to do an interesection!!! :o(
- these were created when I first vectorised the manmade shapes, ficed them using the buffer(0) method suggested on stackoverflow. Check areas before and after were the same, so am happy that no inversion of polygons occured. If the self-intersection is a bowtie the buffer(0) method can invert the polygon, apparently.
- these did not show up when I did the dissolve
- went back to ppp13 to fix invalid polygons, and put in checks after every shape operation to check for anymore
- After first fix, seemed to remain as stable valid polygons.
- reviewed the manual labels where there was disagreement between my label and others' label. Now got a definitive label of 1,2,3 and combined 1&2 to 1 and 3 -> 0 for Yes and No to the question "is parking possible?"
- started work on creating another rectangle fitting algorithm that slides along the rsls in 0.5m intervals. But this will create many rectangles per rsls so I need ato establish a cut off threshold at which to label a TOID as 'has parking' and stop searching.

#### Weds
- finished
- tidied up ppp15 and ppp16
- fit Three Rectangles algorithm and calculated cut off threshold, 0.97 which maximises the number of True labels, compared to the manual labels.
- worked out how many rows to expect in the output: for each toid it is (3 * (number of rsls) * (number of vms polygons) - so sum this total. 


#### problems with Three Rectangle algorithm
- final image shows by choosing only three position, left, middle, right does not try to fit car in the one place that there is a space. 
- Also got some multipolygons were foilage completely cuts off the middle of the rectangle. These don't matter
- Look at third rectangle from top left in last plot above. This is made up of three individual polygons, but will actually only take 2 rows. The 2 left most polygons are a multipolygon geometry that intersects with the top vms, and the final piece is a single polygon that intersects with the bottom vms. This is rare, so not much of a problem.


#### Extensions to work
- scale up to UK
- try a different rectangle fitting algorithm; more intervals/positions/angles
- remove the too small, no roadside edge, roadside edge too narrow filteres before fitting the rectangles?
- USing current theshold of .97 can create more efficient algorithm that stops checking when it finds a rectangle where the overlap proportion is 0.97 or above.
- use labels to train ML algorthim on just the rgb images
- use HSV colour scale as per ONS rather than straight NDVI on IRG images
- use image processing functions in cv2 of just RGB images, get 12.5 cm resolution. Ignore the IRG images. Can use htese tools to segment edges and identify actual vehicles in the image. Though I expect they will still confuse bins/bike sheds

#### Thurs and on  - WRITE UP

## w/c 17/08/2020
- 21/08/2020 final report due

## w/c 24/08/2020
- Monday 24/08/2020 live presention session 09:30 - 10:00
- 15 presentation and live Q&A on code

## Other 

Started thinking about how to tidy up the pipeline from identifying an area to check to producing numbers: something like
- define a geographic area, probably in QGIS
- create AP and TopoLayer versions of these
- collect list of AB+ RD building TOIDs
- collect front gardens TOIDs: TOIDs that intersect with a building and a roadside 
- use Land registry property extent polygons to link buildings & gardens 


## Extensions

### HSV colour space
- resize 50cm NIR and combine with G, B from RGB to do HSV conversion
- convert to HSV colour scale, set threshold for green 
- collect polygons

### image segmenetaion using clustering

